# Linear Programming Example #1
## Lunch Optimization
(https://towardsdatascience.com/linear-programming-and-discrete-optimization-with-python-using-pulp-449f3c5f6e99)

In [1]:
# !conda install -c conda-forge pulp

In [2]:
import pandas as pd
from pulp import *

### How to formulate the optimization problem?
First, we need to create bunches of Python dictionary objects with the information we have from the table. The code is shown below,

In [3]:
# Read the first few rows dataset in a Pandas DataFrame
# Read only the nutrition info not the bounds/constraints
df = pd.read_excel("diet - medium.xls", nrows=17)
df.head()

,Foods,Price/Serving,Serving Size,Calories,Cholesterol (mg),Total_Fat (g),Sodium (mg),Carbohydrates (g),Dietary_Fiber (g),Protein (g),Vit_A (IU),Vit_C (IU),Calcium (mg),Iron (mg)
0,Frozen Broccoli,0.48,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,Frozen Corn,0.54,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
2,Raw Lettuce Iceberg,0.06,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1
3,Baked Potatoes,0.18,1/2 Cup,171.5,0.0,0.2,15.2,39.9,3.2,3.7,0.0,15.6,22.7,4.3
4,Tofu,0.93,1/4 block,88.2,0.0,5.5,8.1,2.2,1.4,9.4,98.6,0.1,121.8,6.2


In [4]:
# Create a list of the food items
food_items = list(df['Foods'])

# Create a dictinary of costs for all food items
costs = dict(zip(food_items, df['Price/Serving']))

# Create a dictionary of calories for all food items
calories = dict(zip(food_items, df['Calories']))

# Create a dictionary of total fat for all food items
fat = dict(zip(food_items, df['Total_Fat (g)']))

# Create a dictionary of carbohydrates for all food items
carbs = dict(zip(food_items, df['Carbohydrates (g)']))

# Create a dictionary of fiber for all food items
fiber = dict(zip(food_items, df['Dietary_Fiber (g)']))

# Create a dictionary of protein for all food items
protein = dict(zip(food_items, df['Protein (g)']))

In [5]:
food_items

['Frozen Broccoli',
 'Frozen Corn',
 'Raw Lettuce Iceberg',
 ' Baked Potatoes',
 'Tofu',
 'Roasted Chicken',
 'Spaghetti W/ Sauce',
 'Raw Apple',
 'Banana',
 'Wheat Bread',
 'White Bread',
 'Oatmeal Cookies',
 'Apple Pie',
 'Scrambled Eggs',
 'Turkey Bologna',
 'Beef Frankfurter',
 'Chocolate Chip Cookies']

In [6]:
# Create a dictionary of protein for all food items
protein = dict(zip(food_items, df['Protein (g)']))
protein

{'Frozen Broccoli': 8.0,
 'Frozen Corn': 2.5,
 'Raw Lettuce Iceberg': 0.2,
 ' Baked Potatoes': 3.7,
 'Tofu': 9.4,
 'Roasted Chicken': 42.2,
 'Spaghetti W/ Sauce': 8.2,
 'Raw Apple': 0.3,
 'Banana': 1.2,
 'Wheat Bread': 2.2,
 'White Bread': 2.3,
 'Oatmeal Cookies': 1.1,
 'Apple Pie': 0.5,
 'Scrambled Eggs': 6.7,
 'Turkey Bologna': 3.9,
 'Beef Frankfurter': 5.4,
 'Chocolate Chip Cookies': 0.9}

Then, we create a LP problem with the method `LpProblemin` PuLP.

In [7]:
prob = LpProblem("Simple Diet Problem", LpMinimize)

Then, we create a dictionary of food items variables with lower bound =0 and category continuous i.e. the optimization solution can take any real-numbered value greater than zero.

> In our mind, we cannot think a portion of food anything other than a non-negative, finite quantity but the mathematics does not know this.

Without an explicit declaration of this bound, the solution may be non-sensical as the solver may try to come up with negative quantities of food choice to reduce the total cost while still meeting the nutrition requirement!

In [8]:
food_vars = LpVariable.dicts("Food", food_items, lowBound=0, cat='Continuous')

Next, we start building the LP problem by adding the main objective function. Note the use of the `lpSum` method.

In [9]:
prob += lpSum([costs[i]*food_vars[i] for i in food_items])

We further build on this by adding calories constraints,

In [10]:
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) >= 800.0
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) <= 1300.0

We can pile up all the nutrition constraints. For simplicity, we are just adding four constraints on fat, carbs, fiber, and protein. The code is shown below,

In [11]:
# Fat
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) >= 20.0, "FatMinimum"
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) <= 50.0, "FatMaximum"

# Carbs
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) >= 130.0, "CarbsMinimum"
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) <= 200.0, "CarbsMaximum"

# Fiber
prob += lpSum([fiber[f] * food_vars[f] for f in food_items]) >= 60.0, "FiberMinimum"
prob += lpSum([fiber[f] * food_vars[f] for f in food_items]) <= 125.0, "FiberMaximum"

# Protein
prob += lpSum([protein[f] * food_vars[f] for f in food_items]) >= 100.0, "ProteinMinimum"
prob += lpSum([protein[f] * food_vars[f] for f in food_items]) <= 150.0, "ProteinMaximum"

And we are done with formulating the problem!
> In any optimization scenario, the hard part is the formulation of the problem in a structured manner which is presentable to a solver.

We have done the hard part. Now, it is the relatively easier part of running a solver and examining the solution.

## Solving the problem and printing the solution
PuLP has quite a few choices of solver algorithms (e.g. COIN_MP, Gurobi, CPLEX, etc.). For this problem, we do not specify any choice and let the program default to its own choice depending on the problem structure.

In [12]:
prob.solve()

1

We can print the status of the solution. Note, although the status is optimal in this case, it does not need to be so. In case the problem is ill-formulated or there is not sufficient information, the solution may be infeasible or unbounded.

In [13]:
# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

Status: Optimal


The full solution contains all the variables including the ones with zero weights. But to us, only those variables are interesting which have non-zero coefficients i.e. which should be included in the optimal diet plan. So, we can scan through the problem variables and print out only if the variable quantity is positive.

In [14]:
for v in prob.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

Food_Frozen_Broccoli = 6.9242113
Food_Scrambled_Eggs = 6.060891
Food__Baked_Potatoes = 1.0806324


So, the optimal solution is to eat 6.923 servings of frozen broccoli, 6.06 servings of scrambled eggs and 1.08 servings of a baked potato!